## Get image paths and lables

In [ ]:
# import json
# import os
# import matplotlib.pyplot as plt
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# import keras
# import random

# from sklearn.metrics import accuracy_score, precision_score, recall_score
# from sklearn.model_selection import train_test_split
# from tensorflow.keras import layers, losses
# from tensorflow.keras.datasets import fashion_mnist
# from tensorflow.keras.models import Model

In [16]:
import matplotlib.pyplot as plt

In [1]:
import tensorflow as tf

2025-05-06 16:07:36.365220: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import os
from sklearn.model_selection import train_test_split

# 
# from PIL import Image
# import numpy as np

def get_image_label_pairs(root_dir):
    image_paths = []
    labels = []

    for root, _, files in os.walk(root_dir):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg')):
                full_path = os.path.join(root, file)
                rel_path = os.path.relpath(full_path, root_dir)
                label = os.path.dirname(rel_path).replace(os.sep, '/')  # Use '/' for consistency
                image_paths.append(full_path)
                labels.append(label)

    return image_paths, labels


In [3]:
path = "../full_scores/"

In [21]:
img_path, img_labels = get_image_label_pairs(path)

# Sampling does not work — needs rewrite

## Sampling

In [4]:
# import tensorflow as tf
import hashlib

def get_deterministic_seed(path, index):
    # Create a hash-based seed from the file path and repetition index
    path_str = path.numpy().decode('utf-8')  # for py_function
    key = f"{path_str}_{index}"
    seed = int(hashlib.sha256(key.encode('utf-8')).hexdigest(), 16) % (2**31)
    return seed

# CHANGE WHITENESS
def load_valid_random_crop(path, index, crop_size=(448, 448), retries=10, whiteness_threshold=0.90):
    def _load(path_str):
        # Use py_function to apply Python-based deterministic logic
        seed = get_deterministic_seed(path_str, index)
        tf.random.set_seed(seed)

        img = tf.io.read_file(path_str)
        img = tf.image.decode_jpeg(img, channels=3)
        img_shape = tf.shape(img)

        crop_h, crop_w = crop_size
        img = tf.image.resize_with_crop_or_pad(img,
                                               tf.maximum(img_shape[0], crop_h),
                                               tf.maximum(img_shape[1], crop_w))

        for attempt in range(retries):
            crop = tf.image.stateless_random_crop(
                img, size=(crop_h, crop_w, 3),
                seed=[seed, attempt]
            )
            crop_float = tf.cast(crop, tf.float32) / 255.0
            mean_brightness = tf.reduce_mean(crop_float)
            if mean_brightness < whiteness_threshold:
                return crop_float
        # Fallback: return last crop even if it's too white
        return crop_float

    # Wrap in tf.py_function for index-aware loading
    return tf.py_function(_load, inp=[path], Tout=tf.float32)


In [5]:
def create_deterministic_crop_dataset(image_paths, labels, repeat_count=5):
    repeated_paths = []
    repeated_labels = []
    crop_indices = []

    for path, label in zip(image_paths, labels):
        for i in range(repeat_count):
            repeated_paths.append(path)
            repeated_labels.append(label)
            crop_indices.append(i)  # each crop gets its index

    path_ds = tf.data.Dataset.from_tensor_slices(repeated_paths)
    label_ds = tf.data.Dataset.from_tensor_slices(repeated_labels)
    index_ds = tf.data.Dataset.from_tensor_slices(crop_indices)

    # Map image loader with deterministic seeding and blank check
    img_ds = tf.data.Dataset.zip((path_ds, index_ds)).map(
        lambda path, idx: load_valid_random_crop(path, idx),
        num_parallel_calls=tf.data.AUTOTUNE
    )

    ae_ds = tf.data.Dataset.zip((img_ds, img_ds))  # for autoencoder
    return ae_ds, label_ds


In [6]:
def split_dataset(image_paths, labels, test_size=0.2, seed=42):
    return train_test_split(
        image_paths, labels, test_size=test_size, random_state=seed, stratify=labels
    )

In [12]:
def get_train_test_datasets(root_dir, repeat_count=5, test_size=0.2):
    image_paths, labels = get_image_label_pairs(root_dir)
    train_paths, test_paths, train_labels, test_labels = split_dataset(image_paths, labels, test_size)

    train_ae_ds, train_label_ds = create_deterministic_crop_dataset(train_paths, train_labels, repeat_count)
    test_ae_ds, test_label_ds = create_deterministic_crop_dataset(test_paths, test_labels, repeat_count)

    train_ds = train_ae_ds.shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
    test_ds = test_ae_ds.batch(32).prefetch(tf.data.AUTOTUNE)

    return train_ds, test_ds, train_label_ds, test_label_ds


In [22]:
image_paths, labels = get_image_label_pairs(path)
train_paths, test_paths, train_labels, test_labels = split_dataset(image_paths, labels, test_size=.2)

In [28]:
len(test_paths)

85